**Librerías**

In [1]:
#| code-fold: false
import pandas as pd
# Import text from url
import requests
import re
# Colores
from termcolor import colored # Solo funciona dentro del notebook
from rich.console import Console # Este paquete logra reflejarlo en el HTML
console = Console(record=True)
# NPL en general
import spacy
# Tokenizacion
import nltk
from nltk.tokenize import word_tokenize
# Stop Words
from nltk.corpus import stopwords
# Steamming and Lemmatization
import nltk
from nltk.stem.snowball import SnowballStemmer
# ENT
from spacy import displacy

In [2]:
%run  "src\counter_to_html.py"

# Preproceso

## Ingesta del documento

Leemos el libro El Quijote.

In [3]:
# Enviar una solicitud GET a la URL
response = requests.get("https://www.gutenberg.org/cache/epub/2000/pg2000.txt")

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    quijote = response.text
    # print("Éxito al leer el texto.")
else:
    print("Error al leer el texto.")


Nos ocupamos únicamente de los primeros seis capítulos de la primera parte de El Quijote.

In [4]:
### Inicio del primer capítulo

# Palabra objetivo
palabra_start = "Primera parte del ingenioso hidalgo don Quijote de la Mancha"

# Expresión regular para capturar todo lo que va antes de un patrón de  búsqueda
patron = re.compile(rf"{palabra_start}\s*(.*)", re.DOTALL)

coincidencia = patron.search(quijote)

if coincidencia:
    quijote = coincidencia.group(1) # 0 incluye la palabra del search, 1 no lo incluye
    console.print("[bold]Inicio del primer capítulo:\n\n[/bold]", quijote[:251])
    # print("Inicio del texto:\n\n", quijote[:251])
else:
    console.print("[bold]Inicio del texto:\n\n[/bold]", quijote[:251])
    

### Inicio del sépimo capítulo (y final del sexto)
palabra_end = "Capítulo VII"

# patron = re.compile(rf"(.*)\b{re.escape(palabra_end)}\b", re.DOTALL) # .* es codicioso ("greedy"): captura la mayor cantidad posible de caracteres antes de la coincidir con la palabra, captura hasta la última aparición de la palabra buscada
patron = re.compile(rf"(.*?)\b{re.escape(palabra_end)}\b", re.DOTALL) # .* es no codicioso ("lazy"): captura la menor cantidad posible de caracteres y evitar que el resto del texto cooincida, se detiene en la primera coincidencia

coincidencia = patron.search(quijote)

if coincidencia:
    quijote = coincidencia.group(0)
    console.print("[bold]Final del sexto capítulo:\n\n[/bold]", quijote[-231:])
    quijote = coincidencia.group(1) # Elimino el patrón de búsqueda
else:
    console.print("[bold]Final del texto:\n\n[/bold]", quijote[-231:])


Inicio del primer capítulo:

 Capítulo primero. Que trata de la condición y ejercicio del famoso hidalgo
don Quijote de la Mancha

En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho
tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua

Final del sexto capítulo:

 o el cura en oyendo el nombre— si tal libro hubiera
mandado quemar; porque su autor fue uno de los famosos poetas del mundo, no
sólo de España, y fue felicísimo en la tradución de algunas fábulas de
Ovidio.




Capítulo VII

## Carga del modelo

Descarga y carga del modelo de lenguaje a través de la librería spaCy

In [5]:
# !python -m spacy download es_core_news_md

In [6]:
#| include: false
# import es_core_news_md
# nlp = es_core_news_md.load()

In [7]:
import spacy
nlp = spacy.load("es_core_news_md")

## Fragmentación de la información

Podemos dividir el texto en párrafos.

In [8]:
#| layout-ncol: 3
for parrafo in list(nlp(quijote).sents)[:3]: # No es necesario pasarlo a una lista, solo para coger un número de ejemplos
    console.print("[blue]\nSiguiente párrafo:\n\n[/blue]", parrafo)
    #print(parrafo)

Siguiente párrafo:

 Capítulo primero.

Siguiente párrafo:

 Que trata de la condición y ejercicio del famoso hidalgo
don Quijote de la Mancha

En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho
tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua,
rocín flaco y galgo corredor.

Siguiente párrafo:

 Una olla de algo más vaca que carnero,
salpicón las más noches, duelos y quebrantos los sábados, lantejas los
viernes, algún palomino de añadidura los domingos, consumían las tres
partes de su hacienda.

Y los párrafos en palabras.

In [9]:
for parrafo in list(nlp(quijote).sents)[:1]:
    console.print("[blue]\nSiguiente párrafo:\n[/blue]")
    # print(colored("\nSiguiente párrafo:\n",'blue'))
    for palabra in parrafo:
        print(palabra)

Siguiente párrafo:

Capítulo
primero
.


# Pipeline NPL

In [10]:
#| code_fold: false
text = quijote

## Tokenization

In [11]:
tokens = word_tokenize(text)
print(tokens[:10])

['Capítulo', 'primero', '.', 'Que', 'trata', 'de', 'la', 'condición', 'y', 'ejercicio']


## Stop Words

In [12]:
stop_words = set(stopwords.words('spanish'))
tokens = [w for w in tokens if w.lower() not in stop_words]
print(tokens[:10])

['Capítulo', 'primero', '.', 'trata', 'condición', 'ejercicio', 'famoso', 'hidalgo', 'don', 'Quijote']


## Stemming and Lemmatization

https://spacy.io/models/es  
https://github.com/explosion/spacy-models/releases?q=es_core_web_md&expanded=true

In [13]:
stemmer = SnowballStemmer("spanish")
tokens = [stemmer.stem(palabra) for palabra in tokens]
tokens[:10]

['capitul',
 'primer',
 '.',
 'trat',
 'condicion',
 'ejercici',
 'famos',
 'hidalg',
 'don',
 'quijot']

## POS Tagging

Part of speech.

Etiqueta las palabras de un texto determinado según sus respectivos tipos de palabras, como sustantivos, adjetivos, adverbios y verbos.

<https://spacy.io/usage>

La librería spaCy se basa en modelos de aprendizaje automático que se entrenaron con grandes cantidades de datos de texto etiquetados.

Las etiquetas que spaCy utiliza se basan en el trabajo realizado por Universal Dependencies, un repositorio común que se puede utilizar para entrenar modelos como spaCy. La página de Dependencias universales tiene información sobre [los corpus disponibles para cada idioma]<https://universaldependencies.org/>.

Las etiquetas que spaCy para un token podemos encontrarlas en <https://spacy.io/api/token#attributes>. Algunas de ellas son:

In [14]:
#| code-fold: false
propiedades = ['text', 'lang_', 'is_digit', 'is_lower', 'is_upper', 'is_sent_start', 'is_sent_end',
               'like_email', 'like_url', # Espero no encontrar ninguna de estas etiquetas en El Quijote
               'sentiment', 'sent']

In [15]:
#| code-fold: false
#| collapse: true
ejemplo_01 = nlp(text)[0]

for attr in propiedades:
    print("obj.%s = %r" % (attr, getattr(ejemplo_01, attr)))

obj.text = 'Capítulo'
obj.lang_ = 'es'
obj.is_digit = False
obj.is_lower = False
obj.is_upper = False
obj.is_sent_start = True
obj.is_sent_end = False
obj.like_email = False
obj.like_url = False
obj.sentiment = 0.0
obj.sent = Capítulo primero.


In [16]:
ejemplo_02 = nlp(text)[:15]

# Encabezado
print(f'{"text":<15} {"lemma_":<15} {"pos_":<15}{"tag_":<15}{"dep_":<15}{"shape_":<15}{"is_alpha":<15}{"is_stop":<15}')
print("="*15*8)

# Loop que imprime los datos en formato de tabla
for token in ejemplo_02:
    print(f'{token.text:<15} {token.lemma_:<15} {token.pos_:<15} {token.tag_:<15} {token.dep_:<15} {token.shape_:<15} {token.is_alpha:<15} {token.is_stop:<15}')


text            lemma_          pos_           tag_           dep_           shape_         is_alpha       is_stop        
Capítulo        Capítulo        PROPN           PROPN           ROOT            Xxxxx           1               0              
primero         primero         ADJ             ADJ             amod            xxxx            1               1              
.               .               PUNCT           PUNCT           punct           .               0               0              
Que             que             SCONJ           SCONJ           mark            Xxx             1               1              
trata           tratar          VERB            VERB            ROOT            xxxx            1               1              
de              de              ADP             ADP             case            xx              1               1              
la              el              DET             DET             det             xx              1            

## Selección de nombres

In [147]:
nlp(text).ents[2]

la Mancha

In [17]:
nouns = []
for token in nlp(text).ents:
    if token.label_  == 'PER':
        nouns.append(token.lemma_)

nouns_tally = Counter(nouns)

df = pd.DataFrame(nouns_tally.most_common(), columns=['noun', 'count'])
df[:100]

,noun,count
0,Andrés,7
1,Nicolás,5
2,señor caballero,5
3,Amadís de Gaula,4
4,Amadís,4
...,...,...
95,Ténganse,1
96,Capítulo VI,1
97,Tomad,1
98,hízolo,1


In [18]:
nouns = []
for token in nlp(text):
    if token.pos_ == 'NOUN':
        nouns.append(token.lemma_)

nouns_tally = Counter(nouns)

df = pd.DataFrame(nouns_tally.most_common(), columns=['noun', 'count'])
df[:100]

,noun,count
0,caballero,61
1,don,56
2,libro,48
3,señor,38
4,cura,33
...,...,...
95,hombre,5
96,batalla,5
97,gigante,5
98,valor,5


### Visualizer (entity recognizer)

Visualizando el reconocedor de entidades. <https://spacy.io/usage/visualizers#ent>

Usamos dos modelos distintos.

In [19]:
text = "María se fue en 2020 al río Guadiana a pescar peces coloridos mientras Mario iban a por bebida al supermercado Mercado"

colors = {"PER": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
          "LOC": "linear-gradient(90deg, orange, lightblue)"}
options = {#"ents": ["PER"], # si quisiéramos especificar las entidades que queremos mostrar
           "colors": colors}

doc = nlp(text)
#displacy.server(doc, style="ent", options=options, auto_select_port=True)
displacy.render(doc, style="ent", options=options)

In [20]:
text = "María se fue en 2020 al río Guadiana a pescar peces coloridos mientras Mario iban a por bebida al supermercado Mercado"

nlp_en = spacy.load("en_core_web_sm")
doc = nlp_en(text)
displacy.render(doc, style="ent", options=options)

In [21]:
import spacy
from spacy import displacy

text = "Cuando Mario Camacho empezó a trabajar en coches autónomos en Google en 2020, poca gente de fuera de la compañía lo tomó en serio"

doc = nlp(text)
displacy.render(doc)

In [22]:
a=nlp(text).ents[0]

In [23]:
for attr in dir(a):
    if attr != 'doc' and attr != 'sent' and attr == 'label_':
        print("obj.%s = %r" % (attr, getattr(a, attr)))

obj.label_ = 'PER'


In [24]:
#for named_entity in nlp(text).ents:
#    print(named_entity, named_entity.label_)

In [25]:
#import spacy

#nlp = spacy.load("es_core_news_md")
doc = nlp(text)[:10]
for token in doc:
    print(token.text, token.pos_, token.tag_ )

Cuando SCONJ SCONJ
Mario PROPN PROPN
Camacho PROPN PROPN
empezó VERB VERB
a ADP ADP
trabajar VERB VERB
en ADP ADP
coches NOUN NOUN
autónomos ADJ ADJ
en ADP ADP


## NER

Named-Entity Recognition.

Realizamos esta subtarea de NPL mediante la librería spaCy.

In [26]:
#| code-fold: false
documento = nlp(text)

Todas las entidades nombradas se encuentran en la propiedad **document.ents**.

In [27]:
for named_entity in list(documento.ents)[:10]:
    print(named_entity, named_entity.label_)

Mario Camacho PER
Google ORG


In [28]:
import pandas as pd
from collections import Counter
people = []

for named_entity in documento.ents:
    if named_entity.label_ == "PER":
        people.append(named_entity.text)

people_tally = Counter(people)

df = pd.DataFrame(people_tally.most_common(), columns=['character', 'count'])
print(df)

       character  count
0  Mario Camacho      1


In [29]:
# Convertir el Counter a una tabla HTML
html_table = counter_to_html_table(people_tally)

# Mostrar la tabla HTML
display(HTML(html_table))

Elemento,Conteo
Mario Camacho,1




**Etiquetas NER**
Las entidades nombradas son frases que contienen los nombres de personas, organizaciones, ubicaciones, expresiones de tiempo, cantidades, entre otras. Las entidades de interés se almacenan con su etiqueta correspondiente (por ejemplo, ‘PER‘ para personas, ‘LOC‘ para lugares, ‘ORG’ para organizaciones y ‘MISC’ que se emplea como un comodín para las tres anteriores) y sus posiciones en el texto (cable aclarar que el modelo que se utiliza mediante spaCy ya tiene las etiquetas predefinidas).  

In [30]:
#| code-fold: false
getattr(nlp.get_pipe('ner'), 'labels')

('LOC', 'MISC', 'ORG', 'PER')

In [31]:
from IPython.display import Markdown, display
import re

def get_ner_in_context(keyword, document, desired_ner_labels= False):
    
    if desired_ner_labels != False:
        desired_ner_labels = desired_ner_labels
    else:
         # all possible labels
        desired_ner_labels = list(nlp.get_pipe('ner').labels)  
        
    #Iterate through all the sentences in the document and pull out the text of each sentence
    for sentence in document.sents:
        #process each sentence
        sentence_doc = nlp(sentence.text)
        for named_entity in sentence_doc.ents:
            #Check to see if the keyword is in the sentence (and ignore capitalization by making both lowercase)
            if keyword.lower() in named_entity.text.lower()  and named_entity.label_ in desired_ner_labels:
                #Use the regex library to replace linebreaks and to make the keyword bolded, again ignoring capitalization
            
                sentence_text = re.sub('\n', ' ', sentence.text)
                sentence_text = re.sub(f"{named_entity.text}", f"**{named_entity.text}**", sentence_text, flags=re.IGNORECASE)

                display(Markdown('--')) # Si pones --- te sale una línea de izq a drcha
                display(Markdown(f"**{named_entity.label_}**"))
                display(Markdown(sentence_text))

In [32]:
type(doc)

spacy.tokens.span.Span

In [33]:
get_ner_in_context('Sancho', nlp(text), desired_ner_labels = False)

In [34]:
import spacy
nlp = spacy.load("es_core_news_md")
doc = nlp("Apple está pensando en comprar una startup de España por 5.000 €.")
for ent in doc.ents:
    print(ent.text, ent.label_)

Apple ORG
España ORG


In [35]:
doc

Apple está pensando en comprar una startup de España por 5.000 €.

In [36]:
get_ner_in_context

<function __main__.get_ner_in_context(keyword, document, desired_ner_labels=False)>